In [ ]:
import cv2
import os
import random
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
num_to_test = 11

def process_imgs(imgs): #read and resize all images w/ cv2
    # 150 x 150 RGB
    nrows = 150
    ncolumns = 150
    channels = 3

    x = []
    for img in imgs:
        try:
            y = cv2.resize(cv2.imread(img, cv2.IMREAD_COLOR), (nrows, ncolumns), interpolation=cv2.INTER_CUBIC)/255
            x.append(y)
        except Exception as e:
#             print(str(e))
            pass
    return x

# Import and setup data
all_data = []
all_labels = []
for x in range(1,(num_to_test+1)):
    data = ['/kaggle/input/{}/{}/{}'.format(x,x,i) for i in os.listdir('/kaggle/input/{}/{}'.format(x,x))]
    data = process_imgs(data)
    all_data.extend(data) #append contents of data  
    all_labels = np.append(all_labels, [x-1 for i in range(len(data))]) #append x len(data) times
    del data

all_data = np.asarray(all_data)
    
# here X is for data, y is for labels 
print(all_data.shape)
print(all_labels.shape)

from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.25)

y_train = to_categorical(y_train, num_classes=num_to_test)
y_test = to_categorical(y_test, num_classes=num_to_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print("> DONE SETUP")

In [ ]:
## This is our original CNN

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

cnn = Sequential()
# cnn.add(conv_base)
cnn.add(Conv2D(filters=4, kernel_size=(3, 3), padding='same', activation='relu', input_shape=X_train.shape[1:]))

cnn.add(MaxPool2D(pool_size=(2, 2)))
cnn.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'))
cnn.add(MaxPool2D(pool_size=(2, 2)))
cnn.add(Flatten())
cnn.add(Dense(32, activation='relu'))

cnn.add(Dense(num_to_test, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

datagen = ImageDataGenerator(
        featurewise_center=True, # set input mean to 0 over the dataset
        samplewise_center=False, # set each sample mean to 0
        featurewise_std_normalization=True, # divide inputs by std of the dataset
        samplewise_std_normalization=False, # divide each input by its std
        zca_whitening=False, # apply ZCA whitening
        rotation_range=20, # randomly rotate images in the range (degrees, 0 to 180)
#         width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
#         horizontal_flip=True, # randomly flip images
#         vertical_flip=False # randomly flip images
)

datagen.fit(X_train[0:100]) # let's say X_sample is a small-ish but statistically representative sample of your data


In [ ]:
## This is using transfer learning

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionResNetV2

conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))

cnn = Sequential()
cnn.add(conv_base)
cnn.add(Conv2D(filters=4, kernel_size=(3, 3), padding='same', activation='relu', input_shape=X_train.shape[1:]))
cnn.add(Flatten())
cnn.add(Dense(32, activation='relu'))

cnn.add(Dense(num_to_test, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

datagen = ImageDataGenerator(
        featurewise_center=True, # set input mean to 0 over the dataset
        samplewise_center=False, # set each sample mean to 0
        featurewise_std_normalization=True, # divide inputs by std of the dataset
        samplewise_std_normalization=False, # divide each input by its std
        zca_whitening=False, # apply ZCA whitening
        rotation_range=20, # randomly rotate images in the range (degrees, 0 to 180)
#         width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
#         horizontal_flip=True, # randomly flip images
#         vertical_flip=False # randomly flip images
)

datagen.fit(X_train[0:100]) # let's say X_sample is a small-ish but statistically representative sample of your data


In [ ]:
# This section is where the model is fit to the data

BS=32
loss = cnn.fit(X_train, 
    y_train, 
    batch_size=128,
    epochs=50)
evals = cnn.evaluate(X_test, y_test, verbose=1)
print('B4 AUG:\ntest loss: {}, test accuracy: {}'.format(evals[0], evals[1]))

cnn.fit_generator(datagen.flow(X_train, y_train, batch_size=BS),
    validation_data=(X_test, y_test), 
    steps_per_epoch=len(X_train) // BS,
    epochs=25)

evals = cnn.evaluate(X_test, y_test, verbose=1)
print('test loss: {}, test accuracy: {}'.format(evals[0], evals[1]))

In [ ]:
dir(loss.history)
loss.history.keys()
plt.plot(loss.history['accuracy'])
# plt.plot(loss.history['val_accuracy'])
plt.title('Convolutional Neural Network')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
# Cleanup
del all_data
gc.collect()
print("DONE CLEANUP")